# 1. 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [176]:
import pickle

In [245]:
service = Service('chromedriver.exe')
driver = webdriver.Chrome(service = service)

In [193]:
isbn13 = pd.read_pickle('isbn13.pkl')

In [194]:
isbn13

0        9791190710343
1        9791190710350
2        9791190710367
3        9791128858338
4        9788954685603
             ...      
54971    9791185020136
54972    9788997838127
54973    9788956591858
54974    9788987794938
54975    9788995696972
Name: isbn13, Length: 54976, dtype: object

In [246]:
rank_mean = [] # 도서 평균 평점
rank_percentage = [] # 평균 percentage
review_title = [] # 리뷰 제목
review_id = [] # 리뷰 id
review_rank = [] # 리뷰 평점
review_text = [] # 리뷰 
isbn_idx = []


for isbn in isbn13[347:1000]: # 페이지 입력
    url = "http://www.yes24.com/product/search?query={}".format(isbn)
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 페이지에 있는 도서 링크(50개)
    book_links = soup.select('a.gd_name')
    goods = []
    for link in book_links:
        a= link['href'].split('/')[3] 
        goods.append(a)
    for item in goods:
        del(goods[1:])
        driver.get('http://www.yes24.com'+'/Product/Goods/{}'.format(item)) # 상품 페이지
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        try:
            mean = soup.select('div.gd_rating > em.yes_b')[0].text
            rank_mean.append([mean]) # 도서 평균 평점
        except:
            mean = []
            rank_mean.append(mean) # 도서 평균 평점
        rank_p = []
        try:
            for k in range(5):
                rank_ = soup.select('span.barGraphBase')[k].text # 평점 percentage
                rank_p.append(rank_)
            rank_percentage.append(rank_p)
        except:
            rank_ = []
            rank_percentage.append(rank_)
        try:
            review_t = []  # 각각의 책의 리뷰제목 리스트
            review_i = []
            review_r = []
            review_te = []
            for page in range(1,3): # 리뷰 페이지 2개
                driver.get('http://www.yes24.com/Product/communityModules/GoodsReviewList/{}?Type=ALL&_=1648001707174&Sort=2&PageNumber={}'.format(item, page))
                time.sleep(1)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                
                blog_link = []
                for i in range(5): # 리뷰 한 페이지에 있는 5개 리뷰 가져오기
                    try:
                        title = soup.select('span.review_tit')[i].text.strip() # 리뷰 제목
                        review_t.append(title)
                        r_id = soup.select('a.lnk_id')[i].text # 리뷰 id
                        review_i.append(r_id)
                        rank = soup.select('span.review_rating')[i].text.split('\n')[1][-2:] # 리뷰 평점
                        review_r.append(rank)
                        blog_l = soup.select('div.review_lnk a')[i]['href'] # 리뷰 블로그 링크
                        blog_link.append(blog_l)
                    except:
                        pass
                for j in range(5): # 리뷰 한 페이지에 있는 5개 블로그 내용 가져오기
                        driver.get(blog_link[j]) # 리뷰 블로그 링크 들어가기
                        time.sleep(1)
                        html = driver.page_source
                        soup = BeautifulSoup(html, 'html.parser')
                        text = soup.select('div.blogContArea')[0].text # 리뷰 내용 
                        review_te.append(text)
        except:
            pass
        review_title.append(review_t)
        review_id.append(review_i)
        review_rank.append(review_r)
        review_text.append(review_te)
        isbn_idx.append(isbn)

InvalidSessionIdException: Message: invalid session id
Stacktrace:
Backtrace:
	Ordinal0 [0x0112CF43+2608963]
	Ordinal0 [0x010BE8C1+2156737]
	Ordinal0 [0x00FB4190+1065360]
	Ordinal0 [0x00FD5A5D+1202781]
	Ordinal0 [0x00FD6C66+1207398]
	GetHandleVerifier [0x012D3252+1679186]
	GetHandleVerifier [0x0138591C+2410012]
	GetHandleVerifier [0x011C3B41+567361]
	GetHandleVerifier [0x011C2CC6+563654]
	Ordinal0 [0x010C4B5B+2181979]
	Ordinal0 [0x010C9AA8+2202280]
	Ordinal0 [0x010C9B95+2202517]
	Ordinal0 [0x010D3B2C+2243372]
	BaseThreadInitThunk [0x764EFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77847A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77847A4E+238]


In [241]:
col = ['isbn','rank_mean','rank_percentage','review_title','review_id','review_rank','review_text']
user = pd.DataFrame([isbn_idx,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text]).T
user.columns = col
user

,isbn,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text
0,9791160262629,[9.6],"[89%, 11%, 0%, 0%, 0%]","[고양이섬 민박집의 대소동, 평화로운 고양이섬의 미스테리, 귀여운 고양이가 가득한 ...","[나**온, s********y, z**b, 섀*, h***m, c********5...","[5점, 4점, 5점, 4점, 5점, 5점, 5점, 5점, 5점, 5점]",[\n하자키 시에 위치한 고양이섬은 물 반 고기 반이 아닌 그 이름대로 고양이 반 ...
1,9791160262605,[9.4],"[76%, 21%, 3%, 0%, 0%]","[『하자키 목련 빌라의 살인』 모두가 용의자?!, [서평]하자키 목련 빌라의 살인 ...","[블*, 나*, 자*련, y******2, r**2, 북*, M*, 네***버, n...","[4점, 5점, 4점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]",[\n이 소설의 배경이 되는 하자키 목련 빌라의 모습을 그려본다. 하드보일드 작가로...
2,9791160262612,[9.6],"[90%, 10%, 0%, 0%, 0%]","[《진달래 고서점의 사체》 마성의 매력 가득한 코지 미스터리!, 불운이 너무 많다 ...","[지*, a******9, 타*****쥐, 날********삶, 명*********...","[4점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]","[\n\n \n\n""난 도대체 이런 데서 뭘 하고 있는 거지?""\n마코토는 카운터를..."
3,9772508333003,[10.0],[],[],[],[],[]
4,9791156625810,[10.0],"[100%, 0%, 0%, 0%, 0%]",[여기까지가 미래입니다-황인찬 시인은 새를 좋아한다. 이번 시집에는 여러 동물이 등...,[홍*],[5점],[\n여기까지가 미래입니다 / 황인찬 시집 / 아시아 출판사황인찬 시인을 좋아한다....
5,9791191043617,[9.6],"[91%, 8%, 0%, 0%, 1%]","[4살 나오코를 죽인 범인은 과연 누구일까요?, 《백광》 괜찮아, 죽여도 괜찮아.,...","[삶**소, 지*, 분**이, 와**웅, 쭈*, 퍼**더, 나*, t****s, 얼...","[5점, 4점, 5점, 5점, 5점, 5점, 4점, 1점, 5점, 5점]",[\n\n 오랜만에 추리물다운 추리물을 만났다. 불행한 사건이 미스터리물의 기본 전...
6,9791168122185,[9.8],"[93%, 7%, 0%, 0%, 0%]","[오늘 하루 마음을 내어드립니다., 이수부 키친, 오늘 하루 마음을 내어드립니다.,...","[달******s, m****5, 솜***방, 가*길, 푸**기, 7***0, c*...","[5점, 5점, 5점, 5점, 5점, 4점, 5점, 5점, 5점, 5점]","[\n#오늘 하루 마음을 내어드립니다.\n#저자의 요리 철학은 한마디로 제약, 절제..."
7,9791197414275,[10.0],"[100%, 0%, 0%, 0%, 0%]","[구깃구깃한 마음도 기록으로 남겨두면 의미가 있지, 오늘 내가 어떤 마음이었는지 기...","[모****이, t*******6, l********1]","[5점, 5점, 5점]",[\n \n \n\n \n \n어느 새 서른보다 마흔이 가까워진 나이. 하지만 마흔...
8,9788934958154,[9.2],"[62%, 38%, 0%, 0%, 0%]","[『하버드 스퀘어』 다시 돌아갈 수 없는 아름다운 시절에 관하여, [서평]하버드 스...","[블*, 나*, 물*울, 노*타, 보**도, 뻑*, s*******9, 밀**재, ...","[4점, 5점, 4점, 4점, 4점, 4점, 5점, 5점, 5점, 5점]",[\n지나간 시절은 다시 돌아갈 수 없기에 아름다운 건가. 힘든 시절을 겪었어도 지...
9,9788965964896,[9.8],"[90%, 10%, 0%, 0%, 0%]","[위스퍼맨 : 속삭이는 살인자, [서평]위스퍼 맨 - 알렉스 노스, 위스퍼맨, 자극...","[아**아, 나*, 모*, K*l, 늘****기, 보**도, 시**스, s***h,...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]","[\n<어벤저스> 루소 형제가 영화화 진행을 하고 있고,\n뉴욕타임스 베스트셀러, ..."


In [244]:
user.to_pickle('user_336-347.pkl', protocol = 4)

In [243]:
isbn13[347]

'9791191560091'

In [ ]:
user1 = pd.read_pickle('user_0-313.pkl')
user2 = pd.read_pickle('user_314-335.pkl') # 이전코드
user3 = pd.read_pickle('user_336-347.pkl')